In [ ]:
import pandas as pd

In [ ]:
df_fake = pd.read_csv("data/Fake.csv")
df_fake.head()

In [ ]:
df_true = pd.read_csv("data/True.csv")
df_true.head()

In [ ]:
list_ = df_true['title'].values,df_fake['title'].values

In [ ]:
str(list_).split("]")[0:]

In [167]:
title = pd.concat([df_fake['title'],df_true['title']],ignore_index=True)
text = pd.concat([df_fake['text'],df_true['text']],ignore_index=True)


In [172]:
df_true['label'] = [0] * df_true.shape[0]
df_fake['label']  = [1] * df_fake.shape[0]

In [175]:
label = pd.concat([df_fake['label'],df_true['label']],ignore_index=True)

In [284]:
label.shape,title.shape,text.shape

((44898,), (44898,), (44898,))

In [301]:
data = pd.DataFrame(data=title,columns=["title"],index=range(0,title.shape[0]))
data 

,title
0,Donald Trump Sends Out Embarrassing New Year’...
1,Drunk Bragging Trump Staffer Started Russian ...
2,Sheriff David Clarke Becomes An Internet Joke...
3,Trump Is So Obsessed He Even Has Obama’s Name...
4,Pope Francis Just Called Out Donald Trump Dur...
...,...
44893,'Fully committed' NATO backs new U.S. approach...
44894,LexisNexis withdrew two products from Chinese ...
44895,Minsk cultural hub becomes haven from authorities
44896,Vatican upbeat on possibility of Pope Francis ...


In [302]:
data["label"] = label
df = data

In [303]:
df.head()

,title,label
0,Donald Trump Sends Out Embarrassing New Year’...,1
1,Drunk Bragging Trump Staffer Started Russian ...,1
2,Sheriff David Clarke Becomes An Internet Joke...,1
3,Trump Is So Obsessed He Even Has Obama’s Name...,1
4,Pope Francis Just Called Out Donald Trump Dur...,1


In [304]:
df = df.dropna()

In [305]:
df['label'].unique()

array([1, 0], dtype=int64)

In [306]:
df.to_csv("news.csv")

In [307]:
df = pd.read_csv("news.csv")
df.head()

,Unnamed: 0,title,label
0,0,Donald Trump Sends Out Embarrassing New Year’...,1
1,1,Drunk Bragging Trump Staffer Started Russian ...,1
2,2,Sheriff David Clarke Becomes An Internet Joke...,1
3,3,Trump Is So Obsessed He Even Has Obama’s Name...,1
4,4,Pope Francis Just Called Out Donald Trump Dur...,1


In [308]:
x = df.drop('label',axis=1)
y = df["label"]

In [309]:
x.shape

(44898, 2)

In [310]:
y.shape

(44898,)

In [311]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [312]:
message = x.copy()
message 

,Unnamed: 0,title
0,0,Donald Trump Sends Out Embarrassing New Year’...
1,1,Drunk Bragging Trump Staffer Started Russian ...
2,2,Sheriff David Clarke Becomes An Internet Joke...
3,3,Trump Is So Obsessed He Even Has Obama’s Name...
4,4,Pope Francis Just Called Out Donald Trump Dur...
...,...,...
44893,44893,'Fully committed' NATO backs new U.S. approach...
44894,44894,LexisNexis withdrew two products from Chinese ...
44895,44895,Minsk cultural hub becomes haven from authorities
44896,44896,Vatican upbeat on possibility of Pope Francis ...


In [313]:
message.reset_index(inplace=True)
message

,index,Unnamed: 0,title
0,0,0,Donald Trump Sends Out Embarrassing New Year’...
1,1,1,Drunk Bragging Trump Staffer Started Russian ...
2,2,2,Sheriff David Clarke Becomes An Internet Joke...
3,3,3,Trump Is So Obsessed He Even Has Obama’s Name...
4,4,4,Pope Francis Just Called Out Donald Trump Dur...
...,...,...,...
44893,44893,44893,'Fully committed' NATO backs new U.S. approach...
44894,44894,44894,LexisNexis withdrew two products from Chinese ...
44895,44895,44895,Minsk cultural hub becomes haven from authorities
44896,44896,44896,Vatican upbeat on possibility of Pope Francis ...


In [314]:
import nltk
import re
from nltk.corpus import stopwords

In [315]:
from nltk.stem.porter import PorterStemmer
pr = PorterStemmer()

In [355]:
ignore_words = [',','?','/','!','=','+','|','#','&','~','(','{',']','}',')','^','-','_','$']
carpus = []
for i in range(len(message)):
    review = re.sub('[^a-zA-z]',' ',message['title'][i])
    review = review.lower()
    review = review.split()
    review  = [pr.stem(w) for w in review if w not in ignore_words]
    review = ' '.join(review)
    carpus .append(review)


In [317]:
### Vocabulary size
voc_size=5000

In [318]:
carpus

['donald trump send out embarrass new year s eve messag thi is disturb',
 'drunk brag trump staffer start russian collus investig',
 'sheriff david clark becom an internet joke for threaten to poke peopl in the eye',
 'trump is so obsess he even ha obama s name code into hi websit imag',
 'pope franci just call out donald trump dure hi christma speech',
 'racist alabama cop brutal black boy while he is in handcuff graphic imag',
 'fresh off the golf cours trump lash out at fbi deputi director and jame comey',
 'trump said some insan racist stuff insid the oval offic and wit back it up',
 'former cia director slam trump over un bulli openli suggest he s act like a dictat tweet',
 'watch brand new pro trump ad featur so much a kiss it will make you sick',
 'papa john s founder retir figur out racism is bad for busi',
 'watch paul ryan just told us he doesn t care about struggl famili live in blue state',
 'bad news for trump mitch mcconnel say no to repeal obamacar in',
 'watch lindsey g

In [319]:
onhot_re = [one_hot(word,voc_size) for word in carpus]
onhot_re

[[2655,
  2584,
  3087,
  4548,
  4265,
  4039,
  4269,
  2269,
  1786,
  3049,
  3022,
  2198,
  4417],
 [3029, 2414, 2584, 2410, 470, 1020, 2817, 625],
 [233,
  4936,
  1130,
  895,
  1418,
  4363,
  3356,
  980,
  4788,
  3859,
  310,
  1546,
  3226,
  3174,
  4485],
 [2584,
  2198,
  4633,
  4125,
  2967,
  2862,
  4158,
  1744,
  2269,
  868,
  3698,
  4869,
  3298,
  711,
  2377],
 [67, 865, 2615, 3217, 4548, 2655, 2584, 4999, 3298, 4832, 4099],
 [3623, 3016, 1636, 1244, 2049, 244, 4938, 2967, 2198, 3226, 755, 4335, 2377],
 [4026,
  2974,
  3174,
  2108,
  1860,
  2584,
  2930,
  4548,
  3804,
  1947,
  290,
  2771,
  4457,
  4289,
  2304],
 [2584,
  4392,
  217,
  3466,
  3623,
  1107,
  3495,
  3174,
  2479,
  7,
  4457,
  3159,
  912,
  2256,
  1825],
 [4585,
  1281,
  2771,
  1154,
  2584,
  1444,
  2002,
  2289,
  4395,
  3850,
  2967,
  2269,
  4939,
  1986,
  2550,
  3034,
  3921],
 [3729,
  2584,
  4039,
  664,
  2584,
  4125,
  1350,
  4633,
  3382,
  2550,
  1892,
  225

In [320]:
sent_legth = 20

In [321]:
embedd_sequeces = pad_sequences(onhot_re,maxlen=sent_legth,padding='pre')
embedd_sequeces

array([[   0,    0,    0, ..., 3022, 2198, 4417],
       [   0,    0,    0, ..., 1020, 2817,  625],
       [   0,    0,    0, ..., 3226, 3174, 4485],
       ...,
       [   0,    0,    0, ..., 2112,  426, 2119],
       [   0,    0,    0, ...,  865,  592, 2120],
       [   0,    0,    0, ..., 1996, 1020, 3160]])

In [322]:
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_legth))
model.add(LSTM(100))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])
print(model.summary())

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 20, 40)            200000    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               56400     
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________
None


In [323]:
import numpy as np

In [324]:
x = np.array(embedd_sequeces)
y = np.array(y)


In [325]:
x.shape

(44898, 20)

In [326]:
y.shape

(44898,)

In [327]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x ,y, test_size=0.33, random_state=42)

In [328]:
model.fit(X_train,y_train,validation_data=(X_test,y_test),epochs=13,batch_size=64)

Epoch 1/13
471/471 [==============================] - 32s 54ms/step - loss: 0.1867 - accuracy: 0.9208 - val_loss: 0.1354 - val_accuracy: 0.9490
Epoch 2/13
471/471 [==============================] - 23s 48ms/step - loss: 0.0943 - accuracy: 0.9650 - val_loss: 0.1132 - val_accuracy: 0.9551
Epoch 3/13
471/471 [==============================] - 26s 55ms/step - loss: 0.0647 - accuracy: 0.9770 - val_loss: 0.1232 - val_accuracy: 0.9539
Epoch 4/13
471/471 [==============================] - 22s 47ms/step - loss: 0.0484 - accuracy: 0.9826 - val_loss: 0.1396 - val_accuracy: 0.9523
Epoch 5/13
471/471 [==============================] - 21s 44ms/step - loss: 0.0345 - accuracy: 0.9875 - val_loss: 0.1511 - val_accuracy: 0.9532
Epoch 6/13
471/471 [==============================] - 21s 45ms/step - loss: 0.0261 - accuracy: 0.9908 - val_loss: 0.1519 - val_accuracy: 0.9551
Epoch 7/13
471/471 [==============================] - 21s 44ms/step - loss: 0.0197 - accuracy: 0.9935 - val_loss: 0.1812 - val_accuracy:

In [329]:
y_pre = model.predict(X_test)

In [330]:
y_pre

array([[9.9999988e-01],
       [1.2342893e-05],
       [7.9509249e-05],
       ...,
       [9.9999827e-01],
       [9.9972522e-01],
       [8.9978957e-06]], dtype=float32)

In [331]:
word = 'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [332]:
def preprocces_word(word_):
    word_ = re.sub('[^a-zA-z]',' ',word_)
    word_ = word_.lower()
    word_ = word_.split()
    word_  = [pr.stem(w) for w in word_ if w not in ignore_words]
    word_ = ' '.join(word_)
    on_hot_test = [one_hot(word_,voc_size)]
    embedd_sequeces_test = pad_sequences(on_hot_test,maxlen=sent_legth,padding='pre')
    embedd_sequeces_test = np.array(embedd_sequeces_test)
    return embedd_sequeces_test


In [333]:
model.predict(preprocces_word(word))

array([[0.99503416]], dtype=float32)

In [334]:
model.predict(preprocces_word("hous dem aid even see comey letter jason chaffetz tweet"))

array([[0.99999976]], dtype=float32)

In [272]:
from tensorflow.keras.layers import Dropout
## Creating model
embedding_vector_features=40
model=Sequential()
model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_legth))
model.add(Dropout(0.3))
model.add(LSTM(100))
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [335]:
model.predict(preprocces_word("'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'"))

array([[0.99503416]], dtype=float32)

In [336]:
model.predict(preprocces_word("Why the Truth Might Get You Fired"))

array([[0.9999997]], dtype=float32)

In [337]:
model.predict(preprocces_word("Donald Trump Sends Out Embarrassing New Year’s Eve Message"))

array([[0.99999976]], dtype=float32)

In [338]:
preprocces_word("Donald Trump Sends Out Embarrassing New Year’s Eve Message")

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0, 2655,
        2584, 3087, 4548, 4265, 4039, 4269, 2269, 1786, 3049]])

In [347]:
df.loc[df["label"]==0,['title']]

23481    As U.S. budget fight looms, Republicans flip t...
23482    U.S. military to accept transgender recruits o...
23483    Senior U.S. Republican senator: 'Let Mr. Muell...
23484    FBI Russia probe helped by Australian diplomat...
23485    Trump wants Postal Service to charge 'much mor...
                               ...                        
44893    'Fully committed' NATO backs new U.S. approach...
44894    LexisNexis withdrew two products from Chinese ...
44895    Minsk cultural hub becomes haven from authorities
44896    Vatican upbeat on possibility of Pope Francis ...
44897    Indonesia to buy $1.14 billion worth of Russia...
Name: title, Length: 21417, dtype: object

In [353]:
print(df["title"][44897    ])
print(df["label"][44897    ])


Indonesia to buy $1.14 billion worth of Russian jets
0


In [358]:
model.predict(preprocces_word("Indonesia to buy $1.14 billion worth of Russian jets"))

array([[9.989147e-06]], dtype=float32)

In [359]:
model.save("fake_new.h5")